In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn

from sklearn.metrics import log_loss, mean_squared_error
from sklearn.model_selection import KFold, train_test_split

import optuna
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_df = pd.read_csv("../input/mmlm22-men/train.csv")
test_df = pd.read_csv("../input/mmlm22-men/test.csv")

In [ ]:
train_df.drop(columns = ['ID','IDTeams_c_score'], inplace = True)
test_df.drop(columns = ['ID','IDTeams_c_score'], inplace = True)

In [ ]:
train_df

In [ ]:
data = train_df.drop(columns=['Pred'])
target = train_df['Pred']

In [ ]:
params={'reg_alpha': 0.0707367246786973, 'reg_lambda': 0.06928113227468818, 'colsample_bytree': 0.647565718092968, 'subsample': 0.44011277781856156, 'learning_rate': 0.0052795751704346455, 'max_depth': 100, 'num_leaves': 356, 'min_child_samples': 72, 'min_data_per_groups': 51}
params['random_state'] = 48
params['n_estimators'] = 15000 
params['learning_rate'] /= 25
params['cat_smooth'] = params.pop('min_data_per_groups')
params['metric'] = 'rmse'

In [ ]:
train_df.drop(columns= ['Pred'], inplace = True)

In [ ]:
test_df

In [ ]:
preds = np.zeros(test_df.shape[0])
kf = KFold(n_splits=10,random_state=48,shuffle=True)
logloss = []
rmse=[]
n=0
for trn_idx, test_idx in kf.split(train_df,target):
    X_tr,X_val=train_df.iloc[trn_idx],train_df.iloc[test_idx]
    y_tr,y_val=target[trn_idx],target[test_idx]
    model = LGBMRegressor(**params)
    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=500,verbose=False)
    preds+=model.predict(test_df)/kf.n_splits
    rmse.append(mean_squared_error(y_val, model.predict(X_val), squared=False))
    print(n+1,rmse[n])
    logloss.append(log_loss(y_val,model.predict(X_val)))
    print(n+1, logloss[n])
    n+=1

In [ ]:
test_df = pd.read_csv("../input/mmlm22-men/test.csv")

In [ ]:
ans =[]
for i in preds:
    if i<0.15:ans.append(0.0)
    elif i>0.15 and i<0.82: ans.append(0.4)
    else:ans.append(1.0)

In [ ]:
sub = pd.DataFrame()
sub['ID'] = test_df['ID']
sub['Pred'] = preds #ans gives 0.599 : preds give 0.465

In [ ]:
sub.to_csv("sub_MMLM2022_9.csv", index = False)